# Functor

## Outline

* Incentive for Functor

* Functor examples

* Functor laws

In this lesson, we will learn about the Functor type classe and how you can use it.

## Incentive for Functor

Imagine you have a function `add1 :: Num a => a -> a` that adds 1 to an variable that has an instance of the Num type class. But your input data is of type `Maybe a`. Let's look at an example.

In [ ]:
add1 :: Num a => a -> a
add1 n = n + 1

var1, var2 :: Maybe Int
var1 = Just 1
var2 = Nothing

add1ForMaybe :: Num a => (a -> a) -> Maybe a -> Maybe a
add1ForMaybe f Nothing = Nothing
add1ForMaybe f (Just n) = Just (f n)

print $ add1ForMaybe add1 var1
print $ add1ForMaybe add1 var2

But what if we have a custom data type as:
```haskell
data Wrapper a = Empty | Wrapper a deriving Show
```
We solve it similar as the previous example.

In [ ]:
data Wrapper a = Empty | Wrapper a deriving Show

myData :: [Wrapper Int]
myData = [Wrapper 3, Empty, Wrapper 2, Wrapper 1, Empty]

applyFuncToWrapper :: (a -> a) -> Wrapper a -> Wrapper a
applyFuncToWrapper f Empty = Empty
applyFuncToWrapper f (Wrapper n) = Wrapper (f n) 

print $ map (applyFuncToWrapper add1) myData

A concern we can have is that for every data type that puts a variable into a context, if we want to perform similiar operations we need to define such a helper function. This is where the Functor type class comes to help. The definition of the Functor type class is as follows:
```haskell
class Functor f where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
```

The minimal complete definition requires just `fmap`. Let's see how this function can help us to avoid using functions for transformations. We look at an example with the Maybe data type.

In [ ]:
add1 :: Int -> Int
add1 n = n + 1

myData :: [Maybe Int]
myData = [Just 3, Nothing, Just 2, Just 1, Nothing]

print $ map (fmap add1) myData

Another way you can use the `fmap` function is in its infix form where the operator `<$>` is just a synonim for \``fmap`\`.

In [ ]:
print $ add1 <$> (Maybe 1 :: Maybe Int)

Also lists have an instance of Functor type class. Imagine that a list is just a context as Maybe, where in the case of Maybe a value might be missing and in the case of lists the are many possible values. For lists it turns out that fmap is just map so you could re-write the previous print statement for myData to:

In [ ]:
print $ fmap (applyFuncToWrapper add1) myData

## Functor examples

Lets try now to use the Wrapper code example by making the Wrapper type an instance of the Functor type class.

In [ ]:
instance Functor Wrapper where
    (fmap) f Empty = Empty
    (fmap) f (Wrapper n) = Wrapper (f n)

print $ map (fmap add1) myData

We get the exact same result as before. A curious reader may say that we only switched 3 lines of code for the `applyFuncForWrapper` function to another 3 lines of code for the `instance` declaration. But you do not have to come up with names for a transformation function every time you creare a new type. Which makes it also easier to refactor code. And you also get the `<$` operation for free. Also for Haskell types that have an instance of Functor the `<$>` operator works out of the box.

Lets look at another example how you can use the `<$>` operator on a Map which has also an instance of Functor. We will define a data structure with record syntax called Book that contains some book details. Then we will create a Map for 3 books and use the function `describe` on the Map to get a list of book descriptions.

In [ ]:
import qualified Data.Map as Map

data Book = Book {
    price :: Int , 
    yearPublished :: Int ,
    title :: String
} deriving Show

book1 = Book {
    price = 15 ,
    yearPublished = 1997 ,
    title = "Harry Potter and the Philosopher's Stone"
}

book2 = Book {
    price = 17 ,
    yearPublished = 1998 ,
    title = "Harry Potter and the Chamber of Secrets"
}

book3 = Book {
    price = 19 ,
    yearPublished = 1999 ,
    title = "Harry Potter and the Prisoner of Azkaban"
}

books :: Map.Map Int Book
books = Map.fromList $ zip [1..3] [book1, book2, book3]

describe :: Book -> String
describe myBook = "The book " ++ title myBook ++
                   " was published in " ++ show (yearPublished myBook) ++
                   " and costs " ++ show (price myBook) ++ "$."

mapM_ print $ describe <$> books

We see that the `<$>` operator by default applies a function to the values of the Map and not its keys. We use the function `mapM_` to print all the elements of the list. The function `mapM_` works like map but throws away the result which is usefull when you want to perform an IO operation. We will talk more about it in the Monad lesson.
```haskell
mapM_ :: (Foldable t, Monad m) => (a -> m b) -> t a -> m ()
```

## Functor laws

Functor has 2 laws. They are as follows:

- Identity:<br>`fmap id = id`

- Composition:<br>`fmap (f . g) = fmap f . fmap g`

## Recap

In this lesson we've discussed:

- the motivation for introducing the Functor type class

- examples that show how to use the Functor type class